# "Donkey Republic queries"
> Below are the results of the queries asked by you, Jan :)


- toc:true- branch: master
- badges: true
- comments: true
- author: Enrique Quiroz
- categories: [fastpages, jupyter]

# 1. Task results
In this first part of the notebook I will answer the questions proposed by Jan, assuming that the table called `rentals` has already been created and each of the columns are formatted correctly. In the second section, I execute the queries and you can see that there is more than one workaround to be done in order to execute the queries.

- **Which user_id made the most rentals?**

Considering only finished rentals, teh query is given by: 
```
SELECT user_id, count(distinct id) as '# of rentals' FROM rentals 
GROUP BY user_id ORDER BY count(distinct id) DESC LIMIT 1;
```
- **How many rentals did the bike_id with the most rentals have?**

The `bike_id` with the most rentals (effective rentals, i.e. finished) is obtained by:
```
SELECT bike_id, count(distinct id) as '# of rentals' FROM rentals WHERE state="finished" 
GROUP BY bike_id ORDER BY count(distinct id) DESC LIMIT 1;
```

- **What %-share of the total number of rentals was cancelled?**

The result is obtained just by averaging the column `state`: 
```
SELECT AVG(state='cancelled')*100 FROM rentals;
```

- **What is the average duration of a non-cancelled rental?**

To calculate the average duration, it is necessary to get the difference between the time the rental was finished and the time it was created, then multiply accordingly to obtain the result in days, hours, minutes, etc. The following query shows thee average duration of a non-cancelled rental in minutes: 
```
SELECT  AVG(CAST((JULIANDAY(finished_at)-julianday(created_at))*24*60 as REAL)) as Duration
from rentals WHERE state='finished';
```

# 2. Extended process

In the following, I will describe and answer the queries proposed by Jan. I used SQLite, which might differ slightly from the SQL used at periscope.
## Checking the data
First, it is important to check the data, to see if it is consistent and if there are any odd values that might affect the queries in the future. Answers will be answered below.
### Create table
First I start by creating the table **rentals**, in case it has not been created yet. for that I use the query: 
```
CREATE TABLE IF NOT EXISTS rentals ( id INTEGER PRIMARY KEY, created_at TEXT, finished_at TEXT, state TEXT, user_id INTEGER, bike_id INTEGER);
```
The fields `created_at` and `finished_at` on some SQL could be simply defined with the type `DATETIME` or `TIMESTAMP`, however, due to the formatting of the dates on the `.csv` file, they have to be initialized as `TEXT`, which will be formatted in a few steps below.

In [7]:
import pandas as pd
from pathlib import Path
Path('my_data.db').touch()
import sqlite3
conn = sqlite3.connect('my_data.db')
c = conn.cursor()

c.execute('''CREATE TABLE IF NOT EXISTS rentals 
( id INTEGER PRIMARY KEY,
  created_at TEXT,
  finished_at TEXT,
  state TEXT,
  user_id INTEGER,
  bike_id INTEGER
)''');

### Import csv file into table
It is time to populate the table, and a sample of this one is shown below

In [3]:
import pandas as pd
# load the data into a Pandas DataFrame
rentals = pd.read_csv('/Users/enriqueqs/Downloads/donkey_rentals_assignment - rentals.csv')
# write the data to a sqlite table
rentals.to_sql('rentals', conn, if_exists='append', index = False)
pd.read_sql_query('''SELECT * FROM rentals''', conn).sample(5)

,id,created_at,finished_at,state,user_id,bike_id
1857,2620040,2020-06-01 16:32:12,2020-06-01 19:14:21,finished,1404576,16689
3424,2629600,2020-06-04 9:38:23,2020-06-04 9:46:55,finished,1241684,20149
4206,2634387,2020-06-05 19:31:56,2020-06-05 19:54:46,finished,1397938,17030
905,2615095,2020-05-31 15:04:10,2020-05-31 16:46:44,finished,650934,9593
1465,2617682,2020-06-01 9:53:00,2020-06-01 18:26:08,finished,1372562,15842


### Check for null values
Checking if there is any row in the data with null values (i.e. checking if the data is corrupted

In [4]:
pd.read_sql_query('''SELECT *
FROM rentals
WHERE coalesce(id , created_at, finished_at, state, user_id, bike_id) IS NULL''', conn)

,id,created_at,finished_at,state,user_id,bike_id


### Formating timestamps
The timestamps are in the format `2020-06-04 9:38:23` when they should be in the form `2020-06-04 09:38:23` (hour should be `09` and not only `9`). So, the text has to be formatted:

In [5]:
c.execute('''update rentals set created_at=case when substr(created_at,13,1) =':' then
(substr(created_at,1,11) || '0' || substr(created_at,12,7) ) else created_at end''')
c.execute('''update rentals set finished_at=case when substr(finished_at,13,1) =':' then
(substr(finished_at,1,11) || '0' || substr(finished_at,12,7) ) else finished_at end''')
pd.read_sql_query('''SELECT * FROM rentals''', conn).sample(5)

,id,created_at,finished_at,state,user_id,bike_id
4914,2638869,2020-06-07 08:13:30,2020-06-07 09:11:09,finished,339682,8757
1894,2620262,2020-06-01 17:19:28,2020-06-02 03:01:10,cancelled,1239844,9612
3505,2630092,2020-06-04 12:52:15,2020-06-04 13:39:11,finished,615818,13897
2478,2623826,2020-06-02 16:00:02,2020-06-02 17:22:59,finished,1381080,16463
3517,2630164,2020-06-04 13:23:26,2020-06-04 14:42:37,finished,275402,24100


And now, the dates can be defined as `TIMESTAMP`:

In [6]:
c.execute('''ALTER TABLE rentals RENAME TO _rentals_old''')

c.execute('''CREATE TABLE rentals
( id INTEGER PRIMARY KEY,
  created_at TIMESTAMP,
  finished_at TIMESTAMP,
  state TEXT,
  user_id INTEGER,
  bike_id INTEGER
)''')

c.execute('''INSERT INTO rentals (id, created_at, finished_at, state, user_id, bike_id)
  SELECT id, created_at, finished_at, state, user_id, bike_id
  FROM _rentals_old''')
c.execute('''DROP TABLE _rentals_old''')

pd.read_sql_query('''SELECT * FROM rentals''', conn).sample(5)

,id,created_at,finished_at,state,user_id,bike_id
3027,2627200,2020-06-03 14:40:59,2020-06-03 14:53:11,finished,785770,24144
3643,2630932,2020-06-04 16:53:41,2020-06-04 16:59:02,finished,1399930,4652
1872,2620130,2020-06-01 16:49:53,2020-06-01 17:15:34,finished,831640,8768
978,2615336,2020-05-31 15:43:34,2020-05-31 15:46:19,cancelled,420704,7535
2619,2624692,2020-06-02 18:59:39,2020-06-02 19:32:13,finished,1375662,5124


## Time to answer the questions

### Which `user_id` made the most rentals?
The query used for this purpose is: 
```
SELECT user_id, count(distinct id) as '# of rentals' FROM rentals 
GROUP BY user_id ORDER BY count(distinct id) DESC LIMIT 2
```

In [30]:
pd.read_sql_query('''SELECT user_id, count(distinct id) as '# of rentals' FROM rentals 
GROUP BY user_id ORDER BY count(distinct id) DESC LIMIT 2''', conn, index_col='user_id')

,# of rentals
user_id,
1402314,13
1405150,10
1408906,9


However, the latter result takes into account the canceled rentals. Which one can regard as not _true_ rentals. The result below shows the top 2 user_id with the most rentals by executing the query:
```
SELECT user_id, count(distinct id) as '# of rentals' FROM rentals WHERE state="finished"
GROUP BY user_id ORDER BY count(distinct id) DESC LIMIT 2 
```

In [29]:
pd.read_sql_query('''SELECT user_id, count(distinct id) as '# of rentals' FROM rentals WHERE state="finished"
GROUP BY user_id ORDER BY count(distinct id) DESC LIMIT 2''', conn, index_col='user_id')

,# of rentals
user_id,
1402314,12
1385446,7
1395354,6


## `bike_id` with more rentals
This result is excluding cancelled rentals and the query is:
```
SELECT bike_id, count(distinct id) as '# of rentals' FROM rentals WHERE state="finished" 
GROUP BY bike_id ORDER BY count(distinct id) DESC LIMIT 3;
```

In [28]:
pd.read_sql_query('''SELECT bike_id, count(distinct id) as '# of rentals' FROM rentals WHERE state="finished" 
GROUP BY bike_id ORDER BY count(distinct id) DESC LIMIT 3''', conn,index_col='bike_id')

,# of rentals
bike_id,
15762,10
15751,9
15458,7


## What %-share of the total number of rentals was cancelled?
The query to answer this is:
```
SELECT AVG(state='cancelled')*100 FROM rentals;
```

In [40]:
print('The percentage share of the total number of rentals that was cancelled is: {}%'.format(
    c.execute('''SELECT AVG(state='cancelled')*100 FROM rentals''').fetchall()[0][0]))

The percentage share of the total number of rentals that was cancelled is: 11.12%


## Average duration of a non-cancelled rental
Below it is shown the average duration of a non-cancelled rental in minutes, by executing the query:
```
SELECT  AVG(CAST((JULIANDAY(finished_at)-julianday(created_at))*24*60 as REAL)) as Duration
from rentals WHERE state='finished';
```

In [42]:
pd.read_sql_query('''SELECT  AVG(CAST((JULIANDAY(finished_at)-julianday(created_at))*24*60 as REAL)) as Duration
from rentals WHERE state='finished' ''', conn)

,Duration
0,173.280318
